In [1]:
import networkx as nx
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import StandardScaler

/home/erikh/GNN_test/gnn-venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
design = pd.read_csv('../data/design_matrix_group_de.tsv', sep="\t")
samples = design['sample'].tolist()
y = np.array(design['group'])-1

In [3]:

edgelist = pd.read_csv('../data/processed/sepsis_edgelist_w_values.csv')
edgelist.fillna(0, inplace=True)
edgelist = edgelist[['parent','child'] + samples]
edgelist.set_index(['parent','child'], inplace=True)

scaled_features = StandardScaler(with_mean=False).fit_transform(edgelist.values)
edgelist = pd.DataFrame(scaled_features, index=edgelist.index, columns=edgelist.columns)
edgelist.reset_index(inplace=True)
edgelist

,parent,child,TM_M2012_010,TM_M2012_011,TM_M2012_012,TM_M2012_013,TM_M2012_014,TM_M2012_016,TM_M2012_017,TM_M2012_018,...,TM_M2012_190,TM_M2012_191,TM_M2012_192,TM_M2012_196,TM_M2012_197,TM_M2012_198,TM_M2012_199,TM_M2012_200,TM_M2012_202,TM_M2012_203
0,Apoptosis,Intrinsic Pathway for Apoptosis,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Apoptosis,Regulation of Apoptosis,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Apoptosis,Caspase activation via extrinsic apoptotic sig...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Apoptosis,Apoptotic execution phase,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Hemostasis,Formation of Fibrin Clot (Clotting Cascade),0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8267,Scavenging by Class F Receptors,HYOU1_HUMAN,0.0,0.000000,0.000000,2.381515,1.822727,2.347224,0.000000,0.000000,...,1.658090,0.000000,1.790136,2.466573,1.760683,1.861283,1.963151,2.017270,2.026863,1.951526
8268,XBP1(S) activates chaperone genes,HYOU1_HUMAN,0.0,0.000000,0.000000,2.381515,1.822727,2.347224,0.000000,0.000000,...,1.658090,0.000000,1.790136,2.466573,1.760683,1.861283,1.963151,2.017270,2.026863,1.951526
8269,Assembly of active LPL and LIPC lipase complexes,ANGL3_HUMAN,0.0,1.759549,1.777595,1.675426,1.848508,1.756070,1.701783,0.000000,...,1.740658,1.711269,1.687311,0.000000,1.693959,1.830008,1.947588,1.984524,1.846057,1.873182
8270,NR1H2 & NR1H3 regulate gene expression linked ...,ANGL3_HUMAN,0.0,1.759549,1.777595,1.675426,1.848508,1.756070,1.701783,0.000000,...,1.740658,1.711269,1.687311,0.000000,1.693959,1.830008,1.947588,1.984524,1.846057,1.873182


In [4]:
all_nodes = list(set(edgelist['child'].values.tolist() + edgelist['parent'].values.tolist()))
all_nodes.remove(0)
G = nx.DiGraph()
for node in all_nodes:
    if node in edgelist['child'].values:
        node_weights = edgelist[edgelist['child'] == node][samples].values[0]
    else:
        node_weights = [0]*len(samples)
    G.add_node(node, weight = node_weights)

for row in edgelist.iterrows():
    row = row[1]
    source = row['child']
    target = row['parent']
    if source in G.nodes():
        G.add_edge(source, target)
    
print(G.number_of_nodes())
print(G.number_of_edges())

2219
6960


In [5]:
pd_adj = nx.to_pandas_adjacency(G)
adj = pd_adj.values
edge_index = (adj > 0).nonzero()
row, col = edge_index
coo = np.array(list(zip(row,col))).T
coo

array([[   0,    1,    2, ..., 2218, 2218, 2218],
       [ 564, 1380,  130, ...,  700, 1373, 1972]])

In [6]:
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader


data_list = []
for i in range(len(samples)):
    sample = samples[i]
    x = []
    for node in G.nodes():
        x.append([G.nodes()[node]['weight'][i]])
    data_i = Data(x=torch.tensor(x, dtype=torch.float), edge_index=torch.tensor(coo, dtype=torch.long), y=torch.tensor(y[i], dtype=torch.long))
    data_list.append(data_i)
    
dataloader = DataLoader(data_list, shuffle=True, batch_size=16, drop_last=True)
dataset = dataloader.dataset
data = dataloader.dataset[1]

print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')


Number of nodes: 2219
Number of edges: 6960
Average node degree: 3.14
Has isolated nodes: False
Has self-loops: False
Is undirected: False


In [7]:

from torch.nn import Dropout, Linear, ReLU
from torch_geometric.nn import GCNConv, Sequential, global_mean_pool

class GCN(torch.nn.Module):
    def __init__(self,  num_classes):
        super(GCN, self).__init__()

        self.num_classes =  num_classes
        # hidden layer node features
        self.hidden = 256
        self.model = Sequential("x, edge_index, batch_index", [                
                (GCNConv(-1, self.hidden), 
                    "x, edge_index -> x1"),
                (ReLU(), "x1 -> x1a"),                                     
                (Dropout(p=0.5), "x1a -> x1d"),                                
                (GCNConv(self.hidden, self.hidden), "x1d, edge_index -> x2"),  
                (ReLU(), "x2 -> x2a"),                                         
                (Dropout(p=0.5), "x2a -> x2d"),                                
                (GCNConv(self.hidden, self.hidden), "x2d, edge_index -> x3"),  
                (ReLU(), "x3 -> x3a"),                                         
                (Dropout(p=0.5), "x3a -> x3d"),                               
                (GCNConv(self.hidden, self.hidden), "x3d, edge_index -> x4"),  
                (ReLU(), "x4 -> x4a"),                                         
                (Dropout(p=0.5), "x4a -> x4d"),                                
                (GCNConv(self.hidden, self.hidden), "x4d, edge_index -> x5"),  
                (ReLU(), "x5 -> x5a"),                                         
                (Dropout(p=0.5), "x5a -> x5d"),                                
                (global_mean_pool, "x5d, batch_index -> x6"),                 
                (Linear(self.hidden, self.num_classes), "x6 -> x_out")])  
        
    def forward(self, graph_data):
        x, edge_index, batch = graph_data.x, graph_data.edge_index,graph_data.batch
        x_out = self.model(x, edge_index, batch)

        return x_out

model = GCN(num_classes=2)
model

GCN(
  (model): Sequential(
    (0): GCNConv(-1, 256)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): GCNConv(256, 256)
    (4): ReLU()
    (5): Dropout(p=0.5, inplace=False)
    (6): GCNConv(256, 256)
    (7): ReLU()
    (8): Dropout(p=0.5, inplace=False)
    (9): GCNConv(256, 256)
    (10): ReLU()
    (11): Dropout(p=0.5, inplace=False)
    (12): GCNConv(256, 256)
    (13): ReLU()
    (14): Dropout(p=0.5, inplace=False)
    (15): <function global_mean_pool at 0x7f3e24a86d30>
    (16): Linear(in_features=256, out_features=2, bias=True)
  )
)

In [8]:

optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()
    for data in dataloader:  # Iterate in batches over the training dataset.
        out = model.forward(data) 
        loss = criterion(out, data.y)  # Compute the loss.
        print(loss)
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.
    return loss
         
def test(loader):
     model.eval()
     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.
         out = model(data)  
         pred = out.argmax(dim=1)  # Use the class with highest probability.
         correct += int((pred == data.y).sum())  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.
         
for epoch in range(1, 200):
    loss = train()
    acc = test(dataloader)
    print(f'Epoch {epoch:>3} | Loss: {loss:.2f} | Acc: {acc*100:.2f}%')

tensor(0.6882, grad_fn=<NllLossBackward0>)
tensor(0.6786, grad_fn=<NllLossBackward0>)
tensor(0.6722, grad_fn=<NllLossBackward0>)
tensor(0.6745, grad_fn=<NllLossBackward0>)
tensor(0.7108, grad_fn=<NllLossBackward0>)
tensor(0.7187, grad_fn=<NllLossBackward0>)
tensor(0.6691, grad_fn=<NllLossBackward0>)
tensor(0.6125, grad_fn=<NllLossBackward0>)
Epoch   1 | Loss: 0.61 | Acc: 55.32%
tensor(0.6813, grad_fn=<NllLossBackward0>)
tensor(0.6923, grad_fn=<NllLossBackward0>)
tensor(0.6787, grad_fn=<NllLossBackward0>)
tensor(0.7067, grad_fn=<NllLossBackward0>)
tensor(0.6661, grad_fn=<NllLossBackward0>)
tensor(0.6437, grad_fn=<NllLossBackward0>)
tensor(0.6739, grad_fn=<NllLossBackward0>)
tensor(0.6992, grad_fn=<NllLossBackward0>)
Epoch   2 | Loss: 0.70 | Acc: 52.48%
tensor(0.7062, grad_fn=<NllLossBackward0>)
tensor(0.6797, grad_fn=<NllLossBackward0>)
tensor(0.6971, grad_fn=<NllLossBackward0>)
tensor(0.6948, grad_fn=<NllLossBackward0>)
tensor(0.6715, grad_fn=<NllLossBackward0>)
tensor(0.6380, grad_fn=

KeyboardInterrupt: 